# Fix Provisioning Issues Notebook

This notebook addresses two critical issues identified in the provisioning script logs:
1. Supervisor not starting
2. VisoMaster repository cloning failing due to directory already existing

Let's solve these issues one by one.

## 1. Check current status

First, let's verify our current environment:

In [ ]:
# Check if supervisord is installed
!command -v supervisord || echo "supervisord not installed"

# Check if supervisord.conf exists
!ls -la /etc/supervisor/conf.d/ 2>/dev/null || echo "No supervisor config directory found"
!ls -la /src/supervisord.conf 2>/dev/null || echo "No supervisord.conf found in /src"
!find / -name "supervisord.conf" -type f 2>/dev/null || echo "No supervisord.conf found anywhere"

# Check VisoMaster directory status
!ls -la /VisoMaster 2>/dev/null || echo "VisoMaster directory doesn't exist"

## 2. Fix Supervisor Installation and Configuration

Let's install and configure supervisor properly:

In [ ]:
# Install supervisor if not present
!apt-get update && apt-get install -y supervisor

# Create basic supervisor config if none exists
!mkdir -p /etc/supervisor/conf.d
!if [ ! -f "/src/supervisord.conf" ]; then
    cat > /src/supervisord.conf << 'EOF'
[supervisord]
nodaemon=true
logfile=/logs/supervisord.log
pidfile=/var/run/supervisord.pid
childlogdir=/logs

[unix_http_server]
file=/var/run/supervisor.sock

[rpcinterface:supervisor]
supervisor.rpcinterface_factory=supervisor.rpcinterface:make_main_rpcinterface

[supervisorctl]
serverurl=unix:///var/run/supervisor.sock

[include]
files=/etc/supervisor/conf.d/*.conf
EOF
    echo "Created default supervisord.conf"
fi

# Create directory for logs
!mkdir -p /logs

# Start supervisor
!supervisord -c /src/supervisord.conf

# Check if it's running
!supervisorctl status || echo "Failed to check supervisor status"

## 3. Fix the VisoMaster Repository

Now let's fix the repository cloning issue:

In [ ]:
# Check the current state of VisoMaster
!if [ -d "/VisoMaster" ]; then
    echo "=== VisoMaster content ==="
    ls -la /VisoMaster
    echo "=== Git status (if applicable) ==="
    cd /VisoMaster && git status 2>/dev/null || echo "Not a valid git repository"
fi

In [ ]:
# Ask user for their preference
import ipywidgets as widgets
from IPython.display import display

print("Choose how to handle the existing VisoMaster directory:")
option = widgets.RadioButtons(
    options=[
        ('Delete and re-clone (CAUTION: Will remove all existing files)', 'delete'),
        ('Keep but update (Will run git pull if it\'s a valid repo)', 'update'),
        ('Rename existing and clone fresh copy', 'rename'),
        ('Do nothing (Keep current state)', 'nothing')
    ],
    value='update',
    description='Action:',
    disabled=False
)

display(option)

button = widgets.Button(description="Execute Selected Action")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        print(f"Executing: {option.value}")
        if option.value == 'delete':
            !rm -rf /VisoMaster
            !git clone "https://github.com/remphan1618/VisoMaster.git" "/VisoMaster"
            print("Repository deleted and re-cloned.")
        elif option.value == 'update':
            !cd /VisoMaster && git pull || echo "Failed to update - not a valid git repository"
            print("Repository updated (if it was a valid git repo).")
        elif option.value == 'rename':
            !mv /VisoMaster /VisoMaster_$(date +%Y%m%d%H%M%S)
            !git clone "https://github.com/remphan1618/VisoMaster.git" "/VisoMaster"
            print("Old repository renamed and fresh copy cloned.")
        else:
            print("No action taken.")

button.on_click(on_button_clicked)
display(button, output)

## 4. Update Provisioning Script to Handle Existing Repository

Let's modify the provisioning script to handle the case when the repository already exists:

In [ ]:
# Create a patched provisioning script
!grep -A 5 -B 5 'git clone.*VisoMaster' /logs/provisioning_script.log || echo "Clone command not found in logs"

# Create a patch file for the provisioning script
!cat > /fix_repo_clone.patch << 'EOF'
# Add this to your provisioning script where the cloning happens
if [ -d "/VisoMaster" ]; then
  echo "VisoMaster directory already exists"
  if [ -d "/VisoMaster/.git" ]; then
    echo "Updating existing repository..."
    cd /VisoMaster && git pull
  else
    echo "Directory exists but is not a git repository. Backing up..."
    mv /VisoMaster "/VisoMaster_backup_$(date +%Y%m%d%H%M%S)"
    git clone "https://github.com/remphan1618/VisoMaster.git" "/VisoMaster"
  fi
else
  echo "Cloning VisoMaster repository..."
  git clone "https://github.com/remphan1618/VisoMaster.git" "/VisoMaster"
fi
EOF

print("Created patch file for repository cloning logic.")
print("This patch needs to be applied to your provisioning script.")

## 5. Verify System Status After Fixes

Let's check the system status after our fixes:

In [ ]:
# Check supervisor status
!supervisorctl status || echo "Supervisor not running"

# Check if VisoMaster repo is in good state
!if [ -d "/VisoMaster/.git" ]; then
    echo "VisoMaster is a valid git repository"
    cd /VisoMaster && git status
else
    echo "VisoMaster is not a valid git repository"
fi

# Check provisioning script logs for remaining errors
!grep -i "error\|fatal\|fail" /logs/provisioning_script.log | tail -n 10

## 6. Add Services to Supervisor

Let's add some basic services to supervisor to demonstrate it's working:

In [ ]:
# Create example service config
!mkdir -p /etc/supervisor/conf.d
!cat > /etc/supervisor/conf.d/sshd.conf << 'EOF'
[program:sshd]
command=/usr/sbin/sshd -D
autostart=true
autorestart=true
stdout_logfile=/logs/sshd.log
stderr_logfile=/logs/sshd_err.log
EOF

# Reload supervisor configuration
!supervisorctl update

# Check status again
!supervisorctl status

## 7. Summary of Fixes

We've implemented the following fixes:

1. **Supervisor Issue**:
   - Installed supervisor if missing
   - Created a proper configuration file
   - Started the supervisor daemon
   - Added an example service

2. **VisoMaster Repository Issue**:
   - Created options for handling existing repository
   - Implemented a patch for the provisioning script to handle this scenario
   - Verified repository status

These fixes should address the issues reported in your provisioning log. Update your provisioning script with the patch provided to prevent similar issues in the future.